In [3]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, date

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt

chart_color = px.colors.qualitative.Plotly

In [4]:
if os.name =='nt':
    df_path = ''
else:
    df_path = '/Users/admin/_Work/Data/icanKID'

# df_User = pd.read_csv(
#     os.path.join(df_path,'df_User.csv')
#     )
df_User = pd.read_csv(
    os.path.join(df_path,'df_User_Child_DeviceBrand.csv')
    )
df_Engagement = pd.read_csv(
    os.path.join(df_path, 'df_Engagement.csv')
    )
df_Engagement_updated = pd.read_csv(
os.path.join(df_path, 'df_Engagement_updated.csv')
    )
df_Payment = pd.read_csv(
    os.path.join(df_path, 'df_Payment.csv')
    )
df_fake_user = pd.read_csv(
    os.path.join(df_path, 'df_fake_paid_users.csv')
    )
# df_DeviceBrand = pd.read_csv('/Users/admin/_Work/Data/icanKID/df_DeviceBrand.csv')

In [5]:
# Discard fake user
df_User = df_User[~df_User['UserID'].isin(df_fake_user['UserID'])]
df_Engagement = df_Engagement[~df_Engagement['UserID'].isin(df_fake_user['UserID'])]
df_Payment = df_Payment[~df_Payment['UserID'].isin(df_fake_user['UserID'])]

In [6]:
user_firstuse = df_Engagement_updated.groupby('UserID')['firstuse'].min().reset_index()
pay_user = df_Payment[~df_Payment.gapday.isna()]

instance_user = pay_user[pay_user.gapday == 0][['UserID','gapday','firstpay','firstPaymentType']]
late_user = pay_user[pay_user.gapday != 0][['UserID','gapday','firstpay','firstPaymentType']]

In [7]:
user_min_registration = df_Engagement.groupby('UserID')['nth_day_from_registration'].min().reset_index()
user_min_registration

def hour_diff_between_use_n_pay(condition, base_df, user_df, addition_info_df):
    base_df = base_df[condition]
    base_df = pd.merge(base_df, user_df, on='UserID', how='inner')
    base_df = pd.merge(base_df, addition_info_df, on='UserID', how='left')
    base_df['use_first'] = pd.to_datetime(base_df['firstuse']) > pd.to_datetime(base_df['firstpay'])
    base_df['gap_hour'] = np.abs((pd.to_datetime(base_df['firstpay']) - pd.to_datetime(base_df['firstuse'])).dt.total_seconds()/3600)

    return base_df

first_case = hour_diff_between_use_n_pay(
    user_min_registration.nth_day_from_registration == 0,
    user_min_registration,
    instance_user,
    user_firstuse
    )
first_case

,UserID,nth_day_from_registration,gapday,firstpay,firstPaymentType,firstuse,use_first,gap_hour
0,000db940-3fe1-48c3-817a-bb19d0a47394,0,0.0,2022-06-04 16:12:00.466585+07:00,zalopay,2022-06-04 17:21:52+07:00,True,1.164315
1,00125ee4-9a93-465e-8e7a-556d1d6ca10e,0,0.0,2022-07-08 19:57:17.510282+07:00,commercial,2022-07-08 20:56:17+07:00,True,0.983192
2,001b851c-a541-4f29-a415-8e7f00f265d7,0,0.0,2022-06-13 12:39:34.630996+07:00,promotion,2022-06-13 19:20:01+07:00,True,6.673991
3,0020793a-d0d7-4105-a479-4e2c38dd9eb4,0,0.0,2022-06-22 13:40:09.658398+07:00,zalopay,2022-06-22 13:47:39+07:00,True,0.124817
4,0045bed3-5153-4ec3-bb95-e9206ccd5aea,0,0.0,2022-07-11 14:44:18.191044+07:00,shopeepay,2022-07-11 18:00:09+07:00,True,3.264114
...,...,...,...,...,...,...,...,...
3516,ffb6ae67-04b2-4e85-bd3c-4bd973fd505b,0,0.0,2022-07-29 13:42:59.696739+07:00,commercial,2022-07-29 12:34:52+07:00,False,1.135471
3517,ffbc3cac-9545-40d5-b825-0537aae0cd0f,0,0.0,2022-06-01 11:51:47.708358+07:00,promotion,2022-06-01 14:52:26+07:00,True,3.010637
3518,ffbf13ae-7578-4399-b73a-b3272bb3fded,0,0.0,2022-07-20 11:37:03.468814+07:00,shopeepay,2022-07-20 19:27:59+07:00,True,7.848759
3519,ffc6abe7-110c-4aaa-982f-49a55424951e,0,0.0,2022-09-19 17:15:23.511880+07:00,momo,2022-09-19 19:24:22+07:00,True,2.149580


In [8]:
second_case = hour_diff_between_use_n_pay(
    user_min_registration.nth_day_from_registration > 0,
    user_min_registration,
    late_user,
    user_firstuse
)
second_case

,UserID,nth_day_from_registration,gapday,firstpay,firstPaymentType,firstuse,use_first,gap_hour
0,00140c13-c697-4b6d-8a68-dd3856150917,2,2.0,2022-05-21 17:24:55.738894+07:00,commercial,2022-05-21 15:40:41+07:00,False,1.737427
1,0051fc73-1d4e-43cb-b60c-bfbff1794273,10,245.0,2022-12-07 20:10:01.348148+07:00,in-app-purchase,2022-04-16 09:47:29+07:00,False,5650.375652
2,00545f83-c8b1-48ba-bc2f-8cb3c5ad879f,13,8.0,2022-07-21 11:32:56.933888+07:00,commercial,2022-07-26 12:26:19+07:00,True,120.889463
3,00572320-88c5-4a14-adb2-2c8b9d4e0dd6,9,9.0,2022-07-04 10:35:07.106777+07:00,commercial,2022-07-04 10:20:40+07:00,False,0.240863
4,00728849-7170-4688-a8a1-bfa70f16b7fd,1,1.0,2022-06-14 12:36:40.359230+07:00,commercial,2022-06-14 10:28:28+07:00,False,2.136766
...,...,...,...,...,...,...,...,...
2296,ff560363-5100-416c-b097-c847fba6cf46,1,9.0,2022-08-17 09:44:00.081020+07:00,commercial,2022-08-09 23:00:23+07:00,False,178.726967
2297,ff8f8808-c0b5-4f8c-bee8-5421ca270429,16,11.0,2022-05-27 09:14:42.023027+07:00,commercial,2022-06-01 10:36:06+07:00,True,121.356660
2298,ff96a8b3-f4ed-4d61-adb8-2af285e92837,2,1.0,2022-06-18 16:22:41.143480+07:00,commercial,2022-06-19 07:31:08+07:00,True,15.140793
2299,ffce7f95-1913-4555-a3de-d4434cfa5640,5,5.0,2022-07-21 11:33:56.680182+07:00,commercial,2022-07-21 16:12:24+07:00,True,4.640922


In [9]:
third_case = hour_diff_between_use_n_pay(
    user_min_registration.nth_day_from_registration > 0,
    user_min_registration,
    instance_user,
    user_firstuse
)
third_case

,UserID,nth_day_from_registration,gapday,firstpay,firstPaymentType,firstuse,use_first,gap_hour
0,00326881-9cfd-477f-85bf-380331227209,1,0.0,2022-06-14 20:49:39.759025+07:00,promotion,2022-06-15 08:11:31+07:00,True,11.364234
1,00717daf-dc70-43ce-b3c2-e63d4e0fe395,1,0.0,2022-05-30 16:46:35.329860+07:00,commercial,2022-05-31 19:41:14+07:00,True,26.910742
2,00d863fe-7488-428b-819b-32ad3bd1e618,5,0.0,2022-05-04 21:27:05.715493+07:00,payoo,2022-05-09 16:30:46+07:00,True,115.061190
3,0106046d-1540-42a3-9ca1-a898eac5a040,3,0.0,2022-06-01 16:13:52.767812+07:00,commercial,2022-06-04 19:35:14+07:00,True,75.355898
4,013a8fe9-1117-4f0c-be8d-173e12257ad3,89,0.0,2022-06-07 12:37:58.125668+07:00,promotion,2022-09-04 09:49:36+07:00,True,2133.193854
...,...,...,...,...,...,...,...,...
1169,ff38b469-f04d-4fd6-9a73-87210f5084c8,1,0.0,2022-06-09 20:45:11.348959+07:00,promotion,2022-06-10 00:55:02+07:00,True,4.164070
1170,ff64719a-0f3d-4593-9d9b-d076509e325a,8,0.0,2022-06-10 20:15:22.637706+07:00,promotion,2022-06-18 18:55:40+07:00,True,190.671490
1171,ff8852a0-ae51-406a-ae92-4e8c8651e3f8,4,0.0,2022-06-03 13:18:08.509187+07:00,promotion,2022-06-07 11:34:19+07:00,True,94.269581
1172,ffd44dff-b5e3-4f78-88bc-34ee6cd8c37b,5,0.0,2022-05-23 17:36:44.388231+07:00,commercial,2022-05-28 23:26:23+07:00,True,125.827392


In [13]:
temp = first_case.groupby(by=['firstPaymentType','use_first'])['UserID'].count().reset_index()
fig = ff.create_distplot(
    [first_case[first_case.use_first]['gap_hour'],
     first_case[~first_case.use_first]['gap_hour']
     ], group_labels=['Use first', 'Pay first']
    )
fig.update_xaxes(title='Hour different between first pay and first use')
fig.show()

In [18]:
fig = ff.create_distplot(
    [second_case[second_case.use_first]['gap_hour'],
     second_case[~second_case.use_first]['gap_hour']
     ], group_labels=['Use first', 'Pay first']
    )
fig.update_xaxes(title='Hour different between first pay and first use')
fig.update_layout(xaxis_range=[-1,40])
fig.show()

In [20]:
fig = ff.create_distplot(
    [third_case[third_case.use_first]['gap_hour'],
     ], group_labels=['Use first']
    )
fig.update_xaxes(title='Hour different between first pay and first use')
fig.update_layout(xaxis_range=[-1,100])
fig.show()